### Compute and store filtered and demodulated velocity field along simulated drifters trajectories

In [1]:
import numpy as np
import geopandas as gpd


import xarray as xr
from matplotlib import pyplot as plt
%matplotlib inline

from xhistogram.xarray import histogram
import dask.dataframe as dd

import mitequinox.utils as ut
from mitequinox.plot import *
import mitequinox.parcels as pa
from xmitgcm import llcreader

from scipy import signal
import scipy.ndimage as im

from sympy import Symbol, pi, atan, factor, lambdify

import mitequinox.plot as pl
import mitequinox.sigp as sp
import pyinterp

from fsspec.implementations.local import LocalFileSystem

In [2]:
from dask.distributed import Client, LocalCluster
#
#cluster = LocalCluster()
#
from dask_jobqueue import PBSCluster
cluster = PBSCluster() #processes=7, cores=7
w = cluster.scale(jobs=7
                 )
#
client = Client(cluster)

In [3]:
def convolve(x, h=None, hilbert=False):
    """ Convolve an input signal with a kernel
    Optionaly compute the Hilbert transform of the resulting time series
    
    Parameters
    x : input signal
    h : filter 
    hilbert : True for Hilbert transform to be applied to the filtered signal
    
    Returns
    x_f : filtered signal or hilbert transform of the filtered signal
    """
    x_f = signal.filtfilt(h, [1], x, axis=-1,padlen=0)#
    if hilbert:
        return signal.hilbert(x_f)
    else:
        return x_f
    
def filt_L(v, h,columns,om, hilbert=False):
    """Call convolve function and returns new dataframe with filtered and demodulated signal
    
    Parameters
    v : input dataframe
    h : filter to use as input in convolve
    columns : columns to filter and demodulate in v
    om : central frequency of h
    hilbert : True for Hilbert transform to be applied to the filtered signal
    
    Returns
    vc : dataframe with new columns with filtered and demodulated signal
    """
    vc = v.copy()
    
    for V in columns:
        vc[V+'_hat'] = convolve(vc[V],h=h,hilbert=hilbert)
    time = np.arange(0,vc.index.size*dt,dt)#.compute()
    exp = np.exp(-1j*om*2*np.pi*time)
    for V in columns:
        vc[V+'_demodulated'] = vc[V+'_hat']*exp
    return vc

#### Load variables along simulated  drifters trajectories (with time as index)

In [4]:
root_dir = '/home/datawork-lops-osi/equinox/mit4320/parcels/'
run_name = 'global_T365j_dt1j_dij50'
parcels_index = 'time'

# choose to select time indexed data for now
p = pa.parcels_output(root_dir+run_name, parquets=[parcels_index])
df = p[parcels_index]
df = pa.degs2ms(df) # convert velocities to m/s
df.head()

df = df.persist()

#### Definition of the filter used along each trajectories to keep only the tidal signal

In [6]:
dt = 1/24 # time step in days

tidal_omega = sp.get_tidal_frequencies("M2", "K2","S2","N2","K1","O1","M4","S4")
omega_M2,omega_S2,omega_N2,omega_K2, domega, name = tidal_omega["M2"],tidal_omega["S2"],tidal_omega["N2"],tidal_omega["K2"], .2, "semidiurnal"
omega = (omega_M2+omega_S2)/2 #central frequency

V = ['zonal_velocity','meridional_velocity']
Tw = 30 #filter length
dband=0.2 #half bandwidth

# Generate filter for given parameters : Tw, dband, omega (maximize gain for those choices)
h = sp.generate_filter(omega, T=Tw, dt=dt, bandwidth=dband, normalized_bandwidth=None)

#### Some cleaning before filtering

In [1]:
def remove_duplicates(df):
    """Find duplicates in dataframe and remove them.
    Create a column 'dt_flag', False if the dt is not different from 1/24
    
    ----------------
    Parameters
    df : dataframe from which duplicates must be removed, dataframe
    
    ----------------
    Returns
    df : input dataframe without duplicates and with new column 'dt_flag', dataframe
    """
    df = df.reset_index().drop_duplicates('time').sort_values('time').set_index('time')
    dt_df = df.index[1:]-df.index[:-1]
    df['dt_flag'] = np.insert(dt_df!=np.timedelta64(1,'h'),0,False)
    return df



In [2]:
remove_duplicates?


In [7]:
# Remove trajectories filled with NaN for some partiotions (partition 0 for example), unknown cause
_df = df.dropna().persist()

In [8]:
# Remove duplicates found in dataframe and create flag column to check that dt is always 1/24
df_drop = _df.groupby('trajectory').apply(remove_duplicates
                                   ).drop(columns='trajectory').reset_index().set_index('time').persist()

<ipython-input-8-d07afee968a9>:2: UserWarning: `meta` is not specified, inferred from partial data. Please provide `meta` if the result is unexpected.
  Before: .apply(func)
  After:  .apply(func, meta={'x': 'f8', 'y': 'f8'}) for dataframe result
  or:     .apply(func, meta=('x', 'f8'))            for series result
  df_drop = _df.groupby('trajectory').apply(remove_duplicates


#### Filter and demodulation (cf doc overleaf for formula) : 
Along each trajectory:

1) Apply the filter, h, near the semi-diurnal tidal frequency, $\omega$

2) Demodulation of the filtered signal (*$e^{i\omega t}$)

In [9]:
df_filtered = df_drop.groupby('trajectory').apply(filt_L,h,['zonal_velocity','meridional_velocity']
                                                  ,omega,hilbert=True).persist()

<ipython-input-9-cde957a09447>:1: UserWarning: `meta` is not specified, inferred from partial data. Please provide `meta` if the result is unexpected.
  Before: .apply(func)
  After:  .apply(func, meta={'x': 'f8', 'y': 'f8'}) for dataframe result
  or:     .apply(func, meta=('x', 'f8'))            for series result
  df_filtered = df_drop.groupby('trajectory').apply(filt_L,h,['zonal_velocity','meridional_velocity']


In [10]:
# Reset index
df_filtered = df_filtered.drop(columns='trajectory').reset_index().sort_values('time').set_index('time').persist()

#### Create and store dask dataframe with real and imaginary part of the analytic signal and demodulated signal for velocity fields

In [11]:
for v in ["zonal_velocity","meridional_velocity"]:
    df_filtered[v+"_hat_real"] = df_filtered[v+"_hat"].map_partitions(lambda x : np.real(x)).persist()

    df_filtered[v+"_hat_imag"] = df_filtered[v+"_hat"].map_partitions(lambda x : np.imag(x)).persist()
    df_filtered[v+"_demodulated_real"] = df_filtered[v+"_demodulated"].map_partitions(lambda x : np.real(x)).persist()
    df_filtered[v+"_demodulated_imag"] = df_filtered[v+"_demodulated"].map_partitions(lambda x : np.imag(x)).persist()

In [12]:
for v in ["zonal_velocity","meridional_velocity"]:
    df_filtered = df_filtered.drop(columns=[v+"_hat",v+"_demodulated"]).persist()

In [13]:
# Store filtered demodulated dataframe
# Title is defined as follow : filtered_itide_Tw{filter length}_sd_band{bandwidth}
ds, dirs = pa.load_logs(root_dir, run_name)
pa.store_parquet(dirs["parquets"], df_filtered, overwrite=True, name="filtered_itide_Tw30_sd_band04")

No reindexing
create new archive: /home/datawork-lops-osi/equinox/mit4320/parcels/global_T365j_dt1j_dij50/parquets/filtered_itide_Tw30_sd_band04


'/home/datawork-lops-osi/equinox/mit4320/parcels/global_T365j_dt1j_dij50/parquets/filtered_itide_Tw30_sd_band04'

In [3]:
# Cell to load the filtered signal
root_dir = '/home/datawork-lops-osi/equinox/mit4320/parcels/'
run_name = 'global_T365j_dt1j_dij50'
parcels_index = 'filtered_itide_Tw30_sd_band04'

# choose to select time indexed data for now
p = pa.parcels_output(root_dir+run_name, parquets=[parcels_index])
df = p[parcels_index]
df = pa.degs2ms(df)
df.head()

,trajectory,lat,lon,z,zonal_velocity,meridional_velocity,sea_level,dt_flag,zonal_velocity_hat_real,zonal_velocity_hat_imag,zonal_velocity_demodulated_real,zonal_velocity_demodulated_imag,meridional_velocity_hat_real,meridional_velocity_hat_imag,meridional_velocity_demodulated_real,meridional_velocity_demodulated_imag
time,,,,,,,,,,,,,,,,
2011-11-15,2000384,-68.021919,-19.239584,0.0,0.0,0.0,0.0,False,0.004085,0.000697,0.004085,0.000697,0.002623,0.004498,0.002623,0.004498
2011-11-15,48000573,55.309990,-40.197918,0.0,0.0,0.0,0.0,False,0.017263,-0.016716,0.017263,-0.016716,-0.005707,0.016081,-0.005707,0.016081
2011-11-15,48000631,44.620720,-37.072918,0.0,0.0,0.0,0.0,False,0.006110,-0.020936,0.006110,-0.020936,0.027630,-0.014982,0.027630,-0.014982
2011-11-15,49000118,64.051880,-88.307640,0.0,0.0,0.0,0.0,False,-0.180781,-0.079333,-0.180781,-0.079333,-0.485498,0.112855,-0.485498,0.112855
2011-11-15,49000872,64.252319,-36.253830,0.0,0.0,0.0,0.0,False,0.034166,0.012898,0.034166,0.012898,-0.028217,0.020949,-0.028217,0.020949


In [7]:
# Check dt (if True then dt is not 1/24): 
df[df.dt_flag==True].zonal_velocity.size.compute()

0

In [16]:
#client.restart()
cluster.close()

distributed.client - ERROR - Failed to reconnect to scheduler after 10.00 seconds, closing client
_GatheringFuture exception was never retrieved
future: <_GatheringFuture finished exception=CancelledError()>
asyncio.exceptions.CancelledError
